# Physical Array DOA testing

In [7]:
import numpy as np
import matplotlib.pyplot as plt
import math
import pyroomacoustics as pra
from pyroomacoustics.directivities import (
    DirectivityPattern,
    DirectionVector,
    CardioidFamily,
)
from pyroomacoustics.doa import circ_dist
from scipy.io import wavfile
from playsound import playsound

### Build Anechoic Simulation Room

In [8]:
fs = 48000 # audio sampling frequency - global (set to freq of input file)

room_x = 10 # stage width
room_y = 10 # room length
order = 0 # reflection order
sigma2_n = 5e-7 # microphone noise variance estimate
v_sound = 343.0 # speed of sound in air

m = pra.make_materials(
    ceiling="hard_surface",
    floor="stage_floor",
    east="brickwork",
    west="brickwork",
    north="brickwork",
    south="brickwork",
) # materials of each surface of room, defines signal absorption

room = pra.ShoeBox([room_x, room_y], fs=fs, materials=m, max_order=order, air_absorption=True, ray_tracing=False)

### Place microphones

Places microphones along room wall (doesn't matter)

In [9]:
# design parameters of array
array_loc = [5, 0.1] # x/y location of array center
n_mics = 8 # number of microphones
sep = 0.1 # microphone separation

N = 1024 # fft length - defines N/2 + 1 frequency bands

R = pra.linear_2D_array(center=array_loc, M=n_mics, phi=0, d=sep)
mic_array = pra.Beamformer(R, room.fs, N) # uses omnidirectional mics by default
room.add_microphone_array(mic_array)

### Load Recorded Signals

Stores them in microphone objects

In [10]:
mic_signals_filepath = '../array_samples/two_source_sinusoid/0'
signals_suffix = '-240913_0736.wav'

for i in range(n_mics):
    file_str = mic_signals_filepath + str(i+1) + signals_suffix
    f_sample, audio = wavfile.read(file_str)

    if i == 0:
        mic_signals = np.zeros((n_mics, len(audio)))

    mic_signals[n_mics-1-i, :] = audio

room.mic_array.signals = mic_signals

/var/folders/_f/h8xwd2k15l36zgym21rpns740000gn/T/ipykernel_91398/3956247689.py:6: WavFileWarning: Chunk (non-data) not understood, skipping it.
  f_sample, audio = wavfile.read(file_str)


### DOA to locate source


In [11]:
# short-time fourier transform of signal
X = np.array(
    [
        pra.transform.stft.analysis(signal, N, N // 2).T
        for signal in room.mic_array.signals
    ]
)

# NormMUSIC algorithm selected, recommended over MUSIC
n_source = 2
doa = pra.doa.normmusic.NormMUSIC(R, fs, N, c=v_sound, num_src=n_source, mode='far', azimuth=np.linspace(0, 180, 30)*np.pi/180)
doa.locate_sources(X, freq_bins=np.arange(1, N))
doa_azimuths = doa.azimuth_recon

print("Recovered azimuths:", doa_azimuths * 180.0/np.pi, "degrees")

Recovered azimuths: [117.93103448  99.31034483] degrees
